In [1]:
import scipy.io
import numpy as np
import CMR_IA as cmr
import matplotlib.pyplot as plt
import pickle
from statistics import mean, stdev
import pprint
import import_ipynb
import time
import pandas as pd
import seaborn as sns

np.set_printoptions(edgeitems=20, linewidth=150,precision=4)

In [2]:
with open("../Data/simu6_design.pkl", 'rb') as inp:
    df_study = pickle.load(inp)
    df_test = pickle.load(inp)

In [3]:
s_mat = np.load('../Data/wordpools/ltp_FR_similarity_matrix.npy')

In [28]:
params = cmr.make_default_params()
params.update(
    beta_enc = 0.5,
    beta_rec = 0.8,
    beta_rec_post = 0.2,
    gamma_fc = 0.5,
    s_fc = 0,
    s_cf = 0,
    phi_s = 0,
    phi_d = 0,
    c_thresh = 0.01,
    c_thresh_ass = 0.01,
    a = 2800,
    b = 2.14,
    m = 0,
    n = 1,
    kappa = 0.05,
    lamb = 0.05,
    eta = 0.01,
    nitems_in_accumulator = 10
)
params

{'beta_enc': 0.5,
 'beta_rec': 0.8,
 'beta_rec_new': 0.5,
 'beta_rec_post': 0.2,
 'beta_distract': None,
 'phi_s': 0,
 'phi_d': 0,
 's_cf': 0,
 's_fc': 0,
 'kappa': 0.05,
 'eta': 0.01,
 'omega': 8,
 'alpha': 4,
 'c_thresh': 0.01,
 'c_thresh_ass': 0.01,
 'd_ass': 1,
 'lamb': 0.05,
 'rec_time_limit': 60000.0,
 'dt': 10,
 'nitems_in_accumulator': 10,
 'max_recalls': 50,
 'learn_while_retrieving': False,
 'a': 2800,
 'b': 2.14,
 'm': 0,
 'n': 1,
 'gamma_fc': 0.5,
 'gamma_cf': 0.5}

In [29]:
df_study = df_study.loc[df_study.session == 0]
df_study['study_itemno1'] = [1,3,5,7,9,11]
df_study['study_itemno2'] = [2,4,6,8,10,12]
df_study

,study_itemno1,study_itemno2,study_item1,study_item2,session
0,1,2,HOOP,BLUEJAY,0
1,3,4,SEAGULL,MAN,0
2,5,6,PIKE,SUIT,0
3,7,8,BOLT,HIGHWAY,0
4,9,10,BOULDER,PUDDLE,0
5,11,12,WEED,CLOWN,0


In [30]:
df_test = df_test.loc[df_test.session == 0]
df_test['test_itemno'] = [1]
df_test['correct_ans'] = [2]
df_test

,test_itemno,test_item,correct_ans,lag,session
0,1,HOOP,2,5,0


In [31]:
df_thin = df_test[['session','test_itemno']]
df_thin = df_thin.assign(s_resp=np.nan, s_rt=np.nan)
df_thin

,session,test_itemno,s_resp,s_rt
0,0,1,NaN,NaN


In [32]:
# extarct the session data
sess = 0
pres_mat = df_study.loc[df_study.session==sess,['study_itemno1','study_itemno2']].to_numpy()
pres_mat = np.reshape(pres_mat,(1,len(pres_mat),2))
pres_mat

array([[[ 1,  2],
        [ 3,  4],
        [ 5,  6],
        [ 7,  8],
        [ 9, 10],
        [11, 12]]])

In [33]:
cue_mat = df_thin.loc[df_thin.session==sess,'test_itemno'].to_numpy()
cue_mat

array([1])

In [34]:
sem_mat = s_mat

In [35]:
model = cmr.CMR2(params, pres_mat, sem_mat, source_mat=None,
           rec_mat=None, ffr_mat=None, cue_mat=cue_mat, task='CR', mode='Final')

In [36]:
model.pres_nos

array([[[ 1,  2],
        [ 3,  4],
        [ 5,  6],
        [ 7,  8],
        [ 9, 10],
        [11, 12]]], dtype=int16)

In [37]:
model.cues_nos

array([1], dtype=int16)

In [38]:
model.all_nos_unique

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12], dtype=int16)

In [39]:
model.pres_indexes

array([[[ 0,  1],
        [ 2,  3],
        [ 4,  5],
        [ 6,  7],
        [ 8,  9],
        [10, 11]]])

In [40]:
model.cues_indexes

array([0])

In [41]:
model.nlists

1

In [42]:
model.run_norm_cr_single_sess()

# in model:
# print(self.att_vec[self.all_nos_unique[item_idx]-1])
# print(np.dot(self.c_old[:self.nitems_unique], self.f[:self.nitems_unique].T))

before cue:
 [[0.1687]
 [0.1687]
 [0.1949]
 [0.1949]
 [0.225 ]
 [0.225 ]
 [0.2598]
 [0.2598]
 [0.3   ]
 [0.3   ]
 [0.3464]
 [0.3464]
 [0.4134]
 [0.2   ]]
c_in
: [[0.7071]
 [0.7071]
 [0.    ]
 [0.    ]
 [0.    ]
 [0.    ]
 [0.    ]
 [0.    ]
 [0.    ]
 [0.    ]
 [0.    ]
 [0.    ]
 [0.    ]
 [0.    ]]
after cue
: [[0.6397]
 [0.6397]
 [0.0855]
 [0.0855]
 [0.0987]
 [0.0987]
 [0.114 ]
 [0.114 ]
 [0.1316]
 [0.1316]
 [0.152 ]
 [0.152 ]
 [0.1813]
 [0.0877]]


In [43]:
model.rec_items

[2]

In [44]:
model.rec_times

[2210]

In [45]:
model.f_in

array([0.6397, 0.6397, 0.3117, 0.3117, 0.3248, 0.3248, 0.3447, 0.3447, 0.3717, 0.3717, 0.4064, 0.4064], dtype=float32)

In [26]:
model.M_FC

array([[0.5   , 0.5   , 0.1768, 0.1768, 0.1531, 0.1531, 0.1326, 0.1326, 0.1148, 0.1148, 0.0994, 0.0994, 0.    , 0.    ],
       [0.5   , 0.5   , 0.1768, 0.1768, 0.1531, 0.1531, 0.1326, 0.1326, 0.1148, 0.1148, 0.0994, 0.0994, 0.    , 0.    ],
       [0.    , 0.    , 0.5   , 0.5   , 0.1768, 0.1768, 0.1531, 0.1531, 0.1326, 0.1326, 0.1148, 0.1148, 0.    , 0.    ],
       [0.    , 0.    , 0.5   , 0.5   , 0.1768, 0.1768, 0.1531, 0.1531, 0.1326, 0.1326, 0.1148, 0.1148, 0.    , 0.    ],
       [0.    , 0.    , 0.    , 0.    , 0.5   , 0.5   , 0.1768, 0.1768, 0.1531, 0.1531, 0.1326, 0.1326, 0.    , 0.    ],
       [0.    , 0.    , 0.    , 0.    , 0.5   , 0.5   , 0.1768, 0.1768, 0.1531, 0.1531, 0.1326, 0.1326, 0.    , 0.    ],
       [0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.5   , 0.5   , 0.1768, 0.1768, 0.1531, 0.1531, 0.    , 0.    ],
       [0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.5   , 0.5   , 0.1768, 0.1768, 0.1531, 0.1531, 0.    , 0.    ],
       [0.    , 0.    , 0.    , 

In [47]:
model.M_CF

array([[0.5   , 0.5   , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    ],
       [0.5   , 0.5   , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    ],
       [0.1768, 0.1768, 0.5   , 0.5   , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    ],
       [0.1768, 0.1768, 0.5   , 0.5   , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    ],
       [0.1531, 0.1531, 0.1768, 0.1768, 0.5   , 0.5   , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    ],
       [0.1531, 0.1531, 0.1768, 0.1768, 0.5   , 0.5   , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    ],
       [0.1326, 0.1326, 0.1531, 0.1531, 0.1768, 0.1768, 0.5   , 0.5   , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    ],
       [0.1326, 0.1326, 0.1531, 0.1531, 0.1768, 0.1768, 0.5   , 0.5   , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    ],
       [0.1148, 0.1148, 0.1326, 

In [48]:
recs = model.rec_items
rts = model.rec_times
result = np.column_stack((recs,rts))
df_thin.loc[df_thin.session==sess, ['s_resp','s_rt']] = result
df_simu = df_thin.merge(df_test,on=['session','test_itemno'])
df_simu

,session,test_itemno,s_resp,s_rt,test_item,correct_ans,lag
0,0,1,2.0,2210.0,HOOP,2,5


In [49]:
tmp1 = df_study.study_itemno1.to_numpy()
tmp2 = df_study.study_itemno2.to_numpy()
tmp = np.concatenate((tmp1, tmp2))
tmp = np.sort(tmp)
tmp

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12])

In [50]:
testid = np.searchsorted(tmp, df_simu.test_itemno.item())
corrid = np.searchsorted(tmp, df_simu.correct_ans.item())

In [51]:
df_simu['corr_fin']= model.f_in[corrid]
df_simu['othe_fin'] = np.mean(np.delete(model.f_in, [testid,corrid]))
df_simu

,session,test_itemno,s_resp,s_rt,test_item,correct_ans,lag,corr_fin,othe_fin
0,0,1,2.0,2210.0,HOOP,2,5,0.63972,0.351864
